

---



**Construccion del RAG orientacion vocacional**



---



# **0. Dependencias e Imports**

## 0.1. descargar librerias de Python

In [1]:
!pip install -q \
    pypdf2 \
    sentence-transformers \
    faiss-cpu \
    langchain \
    langchain-community \
    groq \
    python-dotenv \
    pandas \
    tqdm \
    deepeval

# para usar groq API (gratuito) - Obtener key en: https://console.groq.com
import os
os.environ['GROQ_API_KEY'] = 'KEY GROQ'  # Reemplazar con key

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.8/787.8 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [28]:
#para deep eval
os.environ['OPENAI_API_KEY'] = 'KEY OPENIA'

## 0.2. imports necesarios

In [2]:
import json
import re
from pathlib import Path
from typing import List, Dict, Any
import pandas as pd
from tqdm import tqdm
import numpy as np

# Procesamiento de PDFs
import PyPDF2

# embeddings y busqueda vectorial
from sentence_transformers import SentenceTransformer
import faiss

# LLM
from groq import Groq

# Evaluacion con DeepEval
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric
)
from deepeval.test_case import LLMTestCase

## 0.3. Configuracion del RAG

esta seccion se declaran las direcciones de donde se estaran guardando los datos que utilizara que es la base de datos vectorial y los pensums obtenidos en web scrapping y los parametros del RAG asi como el modelo de LLM a utilizar

In [3]:
class Config:

    # directorios
    DATA_DIR = "data"
    RAW_DIR = f"{DATA_DIR}/raw/galileo/pensums"
    PROCESSED_DIR = f"{DATA_DIR}/processed"
    VECTORDB_DIR = f"{DATA_DIR}/vectordb"

    # procesamiento de texto
    CHUNK_SIZE = 1000  # caracteres por chunk (ajustado para pensums)
    CHUNK_OVERLAP = 200  # overlap entre chunks

    # embeddings
    EMBEDDING_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"  # este soporta español
    EMBEDDING_DIM = 384  # dimension del modelo

    # LLM
    LLM_MODEL = "llama-3.3-70b-versatile"  # modelo de groq
    MAX_TOKENS = 2000
    TEMPERATURE = 0.2  # Más deterministico para respuestas sobre pensums

    # Recuperacion
    TOP_K = 4  # numero de chunks a recuperar


# **1. Construccion del RAG**

## 1.1. Carga de los pensums

este metodo nos permite cargar a memoria todos los pensums obtenidos por el web scrapping

In [4]:
def cargar_pensums_galileo() -> List[Dict[str, Any]]:
    pensums_dir = Path(Config.RAW_DIR)

    if not pensums_dir.exists():
        print(f" No existe directorio: {pensums_dir}")
        print(f" PRIMERO SE EJECUTA EL WEB SCRAPPING")
        return []

    pensums_data = []
    json_files = list(pensums_dir.glob("*.json"))

    if not json_files:
        print(f" no se encontraron archivos JSON en: {pensums_dir}")
        return []

    print(f"\n cargando {len(json_files)} pensums de GALILEO")

    for json_file in tqdm(json_files, desc="Cargando JSONs"):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # verificar que no haya error
            if data.get("error"):
                print(f"     {json_file.name} tiene error: {data['error'][:100]}")
                continue

            # verificar que tenga contenido
            if not data.get("texto_completo") and not data.get("ciclos"):
                print(f"     {json_file.name} no tiene contenido")
                continue

            pensums_data.append({
                "carrera": data["carrera"],
                "universidad": "Galileo",
                "tipo": "pensum",
                "texto": data["texto_completo"],
                "metadata_extra": {
                    "url": data.get("url", ""),
                    "materias_totales": data.get("materias_totales", 0),
                    "creditos_totales": data.get("creditos_totales", 0),
                    "ciclos": data.get("ciclos", [])
                },
                "fuente": str(json_file)
            })

            print(f"    {data['carrera']}: {data.get('materias_totales', 0)} materias")

        except Exception as e:
            print(f"    Error en {json_file.name}: {str(e)}")

    print(f"\n Total de pensums cargados: {len(pensums_data)}")

    return pensums_data

## 1.2. division de pensums por cada ciclo

el objetivo de esta funcion es realizar una division de los pensums por cada ciclo, eventualente esto nos sirve para que en la db vectorial pueda estar guardado por chuncks que son unidades de informacion limitadas y accecibles de una forma mas facil por el LLM

In [5]:
def dividir_pensum_por_ciclos(pensum: Dict[str, Any]) -> List[Dict[str, Any]]:
    chunks = []
    ciclos = pensum.get("metadata_extra", {}).get("ciclos", [])

    if ciclos:
        # si tiene estructura de ciclos, crear un chunk por ciclo
        for ciclo in ciclos:
            texto_chunk = f"UNIVERSIDAD GALILEO\n"
            texto_chunk += f"CARRERA: {pensum['carrera']}\n"
            texto_chunk += f"{'='*60}\n\n"
            texto_chunk += f"{ciclo['nombre']}\n"
            texto_chunk += f"Total de créditos del ciclo: {ciclo.get('total_creditos', 0)}\n\n"

            for materia in ciclo["materias"]:
                texto_chunk += f"- {materia['nombre']}"
                if materia.get('codigo'):
                    texto_chunk += f" (Código: {materia['codigo']})"
                texto_chunk += f" - {materia['creditos']} créditos\n"

            chunks.append({
                "texto": texto_chunk,
                "metadata": {
                    "carrera": pensum["carrera"],
                    "universidad": pensum["universidad"],
                    "tipo": "pensum_ciclo",
                    "ciclo": ciclo["nombre"],
                    "fuente": pensum["fuente"]
                }
            })
    else:
        # si no tiene estructura, dividir por tamaño
        chunks_texto = dividir_en_chunks(pensum["texto"], Config.CHUNK_SIZE, Config.CHUNK_OVERLAP)

        for i, chunk_texto in enumerate(chunks_texto):
            chunks.append({
                "texto": chunk_texto,
                "metadata": {
                    "carrera": pensum["carrera"],
                    "universidad": pensum["universidad"],
                    "tipo": "pensum_chunk",
                    "chunk_id": i,
                    "fuente": pensum["fuente"]
                }
            })

    return chunks

## 1.3. divicion en chunks

este metodo ayuda a poder dividir los chunks con un overlap

In [6]:
def dividir_en_chunks(texto: str, chunk_size: int = 1000, overlap: int = 200) -> List[str]:
    chunks = []
    start = 0
    while start < len(texto):
        end = start + chunk_size
        chunk = texto[start:end]

        # intentar cortar en un punto de ruptura natural
        if end < len(texto):
            ultimo_punto = chunk.rfind('.')
            ultimo_salto = chunk.rfind('\n')
            ultimo_guion = chunk.rfind('-')

            punto_corte = max(ultimo_punto, ultimo_salto, ultimo_guion)

            if punto_corte > chunk_size * 0.5:
                chunk = chunk[:punto_corte + 1]
                end = start + len(chunk)

        chunks.append(chunk.strip())
        start = end - overlap

    return [c for c in chunks if len(c) > 50]

## 1.4. Crear los chunks

este apartado hace uso de los anteriores metodos creados para dividir los chunks y finalmente añadirle los metadatos para poder acceder a ellos de una forma directa

In [7]:
def crear_chunks_con_metadata(pensums_data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    crea chunks de todos los pensums con sus metadatos
    """
    chunks_con_metadata = []

    print(f"\n  Dividiendo {len(pensums_data)} pensums en chunks...")

    for pensum in tqdm(pensums_data, desc="Chunking"):
        # dividir por ciclos (mas semantico)
        chunks = dividir_pensum_por_ciclos(pensum)
        chunks_con_metadata.extend(chunks)

    print(f" total de chunks creados: {len(chunks_con_metadata)}")

    return chunks_con_metadata

## 1.5. creacion de la base de datos vectorial

este metodo permite crear los embeddings y finalmente la base de datos vectorial hecha con FAISS

In [8]:
def crear_base_vectorial(chunks_con_metadata: List[Dict[str, Any]]) -> tuple:
    print(f"\n generando embeddings con {Config.EMBEDDING_MODEL}...")

    # cargar modelo de embeddings
    model = SentenceTransformer(Config.EMBEDDING_MODEL)

    # extraer textos
    textos = [chunk["texto"] for chunk in chunks_con_metadata]

    # generar embeddings
    print("   Generando embeddings...")
    embeddings = model.encode(
        textos,
        show_progress_bar=True,
        batch_size=32,
        convert_to_numpy=True
    )

    # Crear indice FAISS
    print("   Creando indice FAISS...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance
    index.add(embeddings.astype('float32'))

    print(f" Base vectorial creada: {index.ntotal} vectores de dimensión {dimension}")

    return index, chunks_con_metadata, model

## 1.6. Guardado de la base de datos vectorial

Guardar la base de datos vectorial con ayuda de los indices FAISS y sus metadatos de forma local

In [9]:
def guardar_base_vectorial(index, chunks_con_metadata: List[Dict[str, Any]]):

    vectordb_dir = Path(Config.VECTORDB_DIR)
    vectordb_dir.mkdir(parents=True, exist_ok=True)

    # guardar indice FAISS
    index_path = vectordb_dir / "faiss_index.bin"
    faiss.write_index(index, str(index_path))
    print(f" Índice FAISS guardado: {index_path}")

    # guardar metadatos
    metadata_path = vectordb_dir / "chunks_metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(chunks_con_metadata, f, ensure_ascii=False, indent=2)
    print(f" Metadatos guardados: {metadata_path}")

    # guardar info del modelo
    config_path = vectordb_dir / "config.json"
    config_info = {
        "embedding_model": Config.EMBEDDING_MODEL,
        "total_chunks": len(chunks_con_metadata),
        "chunk_size": Config.CHUNK_SIZE,
        "chunk_overlap": Config.CHUNK_OVERLAP
    }
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config_info, f, indent=2)
    print(f" Configuración guardada: {config_path}")

# **2. Utilizando el RAG**

## 2.1. Cargar base de datos Vectorial

en este punto la idea principal es que se utilice el RAG por lo tanto comenzamos con un metodo que cargue la DB vectorial al sistema

In [10]:
def cargar_base_vectorial() -> tuple:
    vectordb_dir = Path(Config.VECTORDB_DIR)

    if not vectordb_dir.exists():
        raise FileNotFoundError(f"no existe la base vectorial en: {vectordb_dir}")

    # cargar indice
    index_path = vectordb_dir / "faiss_index.bin"
    if not index_path.exists():
        raise FileNotFoundError(f"no existe el índice FAISS en: {index_path}")

    index = faiss.read_index(str(index_path))

    # cargar metadatos
    metadata_path = vectordb_dir / "chunks_metadata.json"
    with open(metadata_path, 'r', encoding='utf-8') as f:
        chunks_con_metadata = json.load(f)

    # cargar modelo de embeddings
    model = SentenceTransformer(Config.EMBEDDING_MODEL)

    print(f" Base vectorial cargada: {index.ntotal} vectores")

    return index, chunks_con_metadata, model

## 2.2. Buscar contexto

este metodo nos ayuda a que antes de poder hacerle consultas al LLM buscar que contentexto y donde se encuentra la informacion relevante

In [11]:
def buscar_contexto_relevante(
    query: str,
    index,
    chunks_con_metadata: List[Dict[str, Any]],
    embedding_model,
    top_k: int = 4,
    filtro_carreras: List[str] = None
) -> List[Dict[str, Any]]:
    """
    busca chunks relevantes para una consulta
    """
    # generar embedding de la query
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)[0]

    # buscar en FAISS - traer mas candidatos si hay filtros
    k_busqueda = min(top_k * 5 if filtro_carreras else top_k * 2, len(chunks_con_metadata))
    distances, indices = index.search(
        np.array([query_embedding]).astype('float32'),
        k_busqueda
    )

    print(f" Busqueda inicial: {k_busqueda} candidatos")

    # recuperar chunks con scores
    resultados = []
    for idx, distance in zip(indices[0], distances[0]):
        if idx < len(chunks_con_metadata):  # Verificar índice válido
            chunk_data = chunks_con_metadata[idx].copy()
            chunk_data["score"] = float(distance)
            resultados.append(chunk_data)
            print(f"   Chunk {idx}: {chunk_data['metadata']['carrera'][:50]} - Score: {distance:.4f}")

    # aplicar filtros si existen
    if filtro_carreras:
        carreras_normalizadas = [normalizar_nombre_carrera(c) for c in filtro_carreras]
        print(f" Filtrando por carreras: {carreras_normalizadas}")

        resultados_filtrados = []
        for r in resultados:
            carrera_chunk = normalizar_nombre_carrera(r["metadata"]["carrera"])
            print(f"   comparando: '{carrera_chunk}' con {carreras_normalizadas}")

            # Verificar si alguna carrera filtrada está contenida en el nombre del chunk
            if any(carrera_norm in carrera_chunk or carrera_chunk in carrera_norm
                   for carrera_norm in carreras_normalizadas):
                resultados_filtrados.append(r)
                print(f"       Match!")

        resultados = resultados_filtrados
        print(f"   chunks despues de filtrar: {len(resultados)}")

    # Si no hay resultados con filtro, devolver los top sin filtro
    if not resultados and filtro_carreras:
        print("  No se encontraron resultados con filtro, usando búsqueda sin filtro")
        resultados = []
        for idx, distance in zip(indices[0][:top_k], distances[0][:top_k]):
            if idx < len(chunks_con_metadata):
                chunk_data = chunks_con_metadata[idx].copy()
                chunk_data["score"] = float(distance)
                resultados.append(chunk_data)

    # devolver top_k resultados
    return resultados[:top_k]


## 2.3. Normalizar nombres de carreras

este metodo es mas para una ayuda general, en caso de que los nombres de las carreras esten mal escritos o tengan tildes o caracteres que puedan complicar las busquedas es mejor realizar normalizacion antes de cualquier cosa

In [12]:
def normalizar_nombre_carrera(nombre: str) -> str:
    if not nombre:
        return ""

    nombre = nombre.lower()
    # normalizar acentos y caracteres especiales
    nombre = re.sub(r'[áàäâã]', 'a', nombre)
    nombre = re.sub(r'[éèëê]', 'e', nombre)
    nombre = re.sub(r'[íìïî]', 'i', nombre)
    nombre = re.sub(r'[óòöô]', 'o', nombre)
    nombre = re.sub(r'[úùüû]', 'u', nombre)
    nombre = re.sub(r'ñ', 'n', nombre)
    # remover puntuacion
    nombre = re.sub(r'[^\w\s]', '', nombre)
    # normalizar espacios
    nombre = re.sub(r'\s+', ' ', nombre)
    return nombre.strip()

## 2.4. crear prompt para el LLM

metodo para crear el promt que utilizara el LLM de llama para poder realizar la generacion y comprencion de texto basandose en la base de datos vectorial creada

In [13]:
def crear_prompt_rag(
    query: str,
    contexto_chunks: List[Dict[str, Any]],
    carreras_recomendadas: List[str] = None
) -> str:

    # construir contexto
    contexto_texto = ""
    for i, chunk in enumerate(contexto_chunks, 1):
        metadata = chunk["metadata"]
        contexto_texto += f"\n--- DOCUMENTO {i} ---\n"
        contexto_texto += f"Universidad: Galileo\n"
        contexto_texto += f"Carrera: {metadata['carrera']}\n"
        if metadata.get('ciclo'):
            contexto_texto += f"Ciclo: {metadata['ciclo']}\n"
        contexto_texto += f"\nContenido:\n{chunk['texto']}\n"

    # construir prompt
    intro = "Eres un asistente de orientación vocacional especializado en la Universidad Galileo de Guatemala.\n\n"

    if carreras_recomendadas:
        intro += f"El estudiante ha recibido las siguientes recomendaciones de carreras:\n"
        for carrera in carreras_recomendadas:
            intro += f"  • {carrera}\n"
        intro += "\n"

    prompt = f"""{intro}Tu rol es responder preguntas sobre pensums de carreras universitarias basándote ÚNICAMENTE en la información proporcionada.

INFORMACIÓN DISPONIBLE (Pensums de Universidad Galileo):
{contexto_texto}

INSTRUCCIONES IMPORTANTES:
- Responde de forma clara, precisa y objetiva
- Usa SOLO la información del contexto proporcionado
- Si la información no está en el contexto, di claramente: "No encuentro esa información en los pensums disponibles"
- NO inventes materias, créditos o ciclos
- Si te preguntan sobre comparaciones, preséntala de forma neutral y objetiva
- Menciona siempre la carrera y ciclo cuando hables de materias específicas
- Sé conversacional pero profesional

PREGUNTA DEL ESTUDIANTE:
{query}

RESPUESTA:"""

    return prompt

## 2.5. generacion de respuestas

consulta al LLM utilizando el prompt anterior

In [14]:
def generar_respuesta(
    query: str,
    contexto_chunks: List[Dict[str, Any]],
    carreras_recomendadas: List[str] = None
) -> str:
    """
    Genera respuesta usando Groq API (Llama 3.1)
    """
    # crear prompt
    prompt = crear_prompt_rag(query, contexto_chunks, carreras_recomendadas)

    # llamar a Groq API
    try:
        api_key = os.environ.get("GROQ_API_KEY")
        if not api_key:
            return " Error: GROQ_API_KEY no configurada"

        client = Groq(api_key=api_key)

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "Eres un asistente de orientación vocacional profesional, objetivo y servicial."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model=Config.LLM_MODEL,
            temperature=Config.TEMPERATURE,
            max_tokens=Config.MAX_TOKENS
        )

        respuesta = chat_completion.choices[0].message.content
        return respuesta

    except Exception as e:
        return f" Error al generar respuesta: {str(e)}"

## 2.6. Construccion de consultas

creacion de consultas del RAG utilizando la db vectorial y las consultas del LLM con groq

In [15]:
def consultar_rag(
    query: str,
    carreras_recomendadas: List[str] = None
) -> Dict[str, Any]:
    try:
        # cargar base vectorial
        index, chunks_metadata, embedding_model = cargar_base_vectorial()

        # buscar contexto relevante
        chunks_relevantes = buscar_contexto_relevante(
            query=query,
            index=index,
            chunks_con_metadata=chunks_metadata,
            embedding_model=embedding_model,
            top_k=Config.TOP_K,
            filtro_carreras=carreras_recomendadas
        )

        if not chunks_relevantes:
            return {
                "respuesta": "No encontré información relevante para tu consulta en los pensums disponibles.",
                "fuentes": [],
                "contexto": [],
                "chunks_utilizados": 0,
                "carreras_filtradas": carreras_recomendadas
            }

        # generar respuesta
        respuesta = generar_respuesta(
            query=query,
            contexto_chunks=chunks_relevantes,
            carreras_recomendadas=carreras_recomendadas
        )

        # compilar fuentes unicas
        fuentes = []
        carreras_vistas = set()
        for chunk in chunks_relevantes:
            carrera = chunk["metadata"]["carrera"]
            if carrera not in carreras_vistas:
                fuentes.append({
                    "universidad": "Galileo",
                    "carrera": carrera,
                    "ciclo": chunk["metadata"].get("ciclo", "N/A")
                })
                carreras_vistas.add(carrera)

        return {
            "respuesta": respuesta,
            "fuentes": fuentes,
            "contexto": [chunk["texto"] for chunk in chunks_relevantes],
            "chunks_utilizados": len(chunks_relevantes),
            "carreras_filtradas": carreras_recomendadas
        }

    except FileNotFoundError as e:
        return {
            "respuesta": f" Error: {str(e)}\n\naun no existe un RAG",
            "fuentes": [],
            "contexto": [],
            "chunks_utilizados": 0,
            "carreras_filtradas": carreras_recomendadas
        }
    except Exception as e:
        return {
            "respuesta": f" Error inesperado: {str(e)}",
            "fuentes": [],
            "contexto": [],
            "chunks_utilizados": 0,
            "carreras_filtradas": carreras_recomendadas
        }

## 2.7. Construir el RAG

metodo para poder ejecutar paso a paso la creacion del RAG en forma de pipeline

In [16]:
def construir_rag_completo():

    print("CONSTRUCCIÓN DEL SISTEMA RAG - PENSUMS GALILEO")

    # 1. Cargar pensums de galileo
    print("PASO 1: Carga de pensums")
    print("-" * 80)

    pensums = cargar_pensums_galileo()

    if not pensums:
        print("\n No hay pensums para procesar")
        return

    # 2. Crear chunks
    print("\n" + "="*80)
    print("PASO 2: Chunking de documentos")
    print("-" * 80)

    chunks_con_metadata = crear_chunks_con_metadata(pensums)

    # 3. Crear base vectorial
    print("\n" + "="*80)
    print("PASO 3: Generacion de embeddings y base vectorial")
    print("-" * 80)

    index, chunks_metadata, model = crear_base_vectorial(chunks_con_metadata)

    # 4. guardar
    print("\n" + "="*80)
    print("PASO 4: Guardando base de datos vectorial")
    print("-" * 80)

    guardar_base_vectorial(index, chunks_metadata)

    # Resumen final
    print("\n" + "-"*40)
    print("CONSTRUCCION COMPLETADA")
    print("-"*40)

    # estadisticas
    carreras_unicas = set(p["carrera"] for p in pensums)

    print(f"""
     Pensums procesados: {len(pensums)}
     Carreras unicas: {len(carreras_unicas)}
     Chunks creados: {len(chunks_metadata)}
     Base vectorial guardada en: {Config.VECTORDB_DIR}

    Carreras incluidas:
    {chr(10).join(f'  • {c}' for c in sorted(carreras_unicas))}

     El sistema RAG está listo para consultas
    """)

# 3. deepeval

el deep eval implementado para el RAG contiene:

* Answer relevancy: que tan relevante es la respuesta
* Faithfulness: Que tan fiel es al contexto recuperado
* Contextual precision: precision del contexto recuperado
* Contextual recall: completitud del contexto

In [40]:
def evaluar_rag_con_deepeval():
    print("\n" + "="*80)
    print("EVALUACIÓN DEL RAG CON DEEPEVAL")
    print("="*80)

    # Casos de prueba
    test_cases = [
        {
            "input": "¿Qué materias de matemáticas tiene Ingeniería de Sistemas?",
            "expected_output": "Matemática I, II, III, IV, V, VI, VII, Álgebra Lineal I y II, Ecuaciones Diferenciales Ordinarias, Estadística Matemática",
            "carreras": ["INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN"]
        },
        {
            "input": "¿Cuántos créditos tiene la carrera de Ingeniería de Sistemas?",
            "expected_output": "227 créditos totales",
            "carreras": ["INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN"]
        },
        {
            "input": "¿Qué materias de programación hay en el primer año?",
            "expected_output": "Informática I, II, III y Ciencias de la Computación I, II",
            "carreras": None
        }
    ]

    deepeval_test_cases = []

    for test in test_cases:
        print(f"\n Evaluando: {test['input']}")

        # ejecutar consulta RAG
        resultado = consultar_rag(
            query=test['input'],
            carreras_recomendadas=test['carreras']
        )

        # crear caso de prueba DeepEval
        test_case = LLMTestCase(
            input=test['input'],
            actual_output=resultado['respuesta'],
            expected_output=test['expected_output'],
            retrieval_context=resultado['contexto']
        )
        deepeval_test_cases.append(test_case)

        print(f" Respuesta: {resultado['respuesta'][:200]}...")
        print(f" Chunks utilizados: {resultado['chunks_utilizados']}")

    # Definir metricas
    print("\n" + "-"*80)
    print("Calculando metricas deepeval...")
    print("-"*80)

    metrics = [
        AnswerRelevancyMetric(threshold=0.7),
        FaithfulnessMetric(threshold=0.7),
        ContextualPrecisionMetric(threshold=0.7),
        ContextualRecallMetric(threshold=0.7)
    ]

    # evaluar
    results = evaluate(
        test_cases=deepeval_test_cases,
        metrics=metrics
    )

    # mostrar resultados
    print("\n" + "="*80)
    print("RESULTADOS DE EVALUACION")
    print("="*80)

    for i, (test, result) in enumerate(zip(test_cases, deepeval_test_cases), 1):
        print(f"\n--- Test Case {i} ---")
        print(f"Pregunta: {test['input']}")
        print(f"Metricas:")
        for metric in metrics:
            score = getattr(result, metric.__class__.__name__.replace('Metric', '').lower(), 'N/A')
            print(f"  • {metric.__class__.__name__}: {score}")

    return results

# 4. Probando el RAG

## 4.1 extraer los pensums de Galileo

In [18]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/raw/
   creating: data/raw/galileo/
   creating: data/raw/galileo/pensums/
  inflating: data/raw/galileo/pensums/ingenieria_administrativa.json  
  inflating: data/raw/galileo/pensums/ingenieria_administrativa.txt  
  inflating: data/raw/galileo/pensums/ingenieria_en_electronica.json  
  inflating: data/raw/galileo/pensums/ingenieria_en_electronica.txt  
  inflating: data/raw/galileo/pensums/ingenieria_en_mecatronica.json  
  inflating: data/raw/galileo/pensums/ingenieria_en_mecatronica.txt  
  inflating: data/raw/galileo/pensums/ingenieria_en_sistemas.json  
  inflating: data/raw/galileo/pensums/ingenieria_en_sistemas.txt  
  inflating: data/raw/galileo/pensums/ingenieria_en_telecomunicaciones.json  
  inflating: data/raw/galileo/pensums/ingenieria_en_telecomunicaciones.txt  
  inflating: data/raw/galileo/pensums/ingenieria_industrial.json  
  inflating: data/raw/galileo/pensums/ingenieria_industrial.txt  
  inflating: data/raw/g

## 4.2. Creando el RAG

In [22]:
construir_rag_completo()

CONSTRUCCIÓN DEL SISTEMA RAG - PENSUMS GALILEO
PASO 1: Carga de pensums
--------------------------------------------------------------------------------

 cargando 24 pensums de GALILEO



Cargando JSONs: 100%|██████████| 24/24 [00:00<00:00, 5525.18it/s]


    Licenciatura en Mercadotecnia: 48 materias
    Licenciatura en Diplomacia y Relaciones Internacionales: 56 materias
    Licenciatura en Enfermería: 16 materias
    Licenciatura en Fisioterapia: 51 materias
    INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN: 51 materias
    Licenciatura en Psicopedagogía: 60 materias
    Ingeniería Industrial: 53 materias
    Licenciatura en Optometría: 49 materias
    Ingeniería Administrativa: 53 materias
    Licenciatura en Gerencia de Negocios y Servicios: 37 materias
    Licenciatura en Medios de Comunicación e Informativos: 48 materias
    INGENIERIA QUIMICA: 46 materias
    Licenciatura en Administración y Microfinanzas: 64 materias
    Licenciatura en Comunicacion y Diseño: 90 materias
    INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN: 48 materias
    Licenciatura en Gestión Empresarial: 40 materias
    Licenciatura en Música: 64 materias
    Licenciatura en Administracion de Empresas: 48 materias
    L


Chunking: 100%|██████████| 24/24 [00:00<00:00, 13322.30it/s]

 total de chunks creados: 249

PASO 3: Generacion de embeddings y base vectorial
--------------------------------------------------------------------------------

 generando embeddings con paraphrase-multilingual-MiniLM-L12-v2...


...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   Generando embeddings...


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

   Creando indice FAISS...
 Base vectorial creada: 249 vectores de dimensión 384

PASO 4: Guardando base de datos vectorial
--------------------------------------------------------------------------------
 Índice FAISS guardado: data/vectordb/faiss_index.bin
 Metadatos guardados: data/vectordb/chunks_metadata.json
 Configuración guardada: data/vectordb/config.json

----------------------------------------
CONSTRUCCION COMPLETADA
----------------------------------------

     Pensums procesados: 24
     Carreras unicas: 24
     Chunks creados: 249
     Base vectorial guardada en: data/vectordb

    Carreras incluidas:
      • INGENIERIA QUIMICA
  • INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN
  • INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN
  • INGENIERÍA EN MECATRÓNICA
  • INGENIERÍA EN TELECOMUNICACIONES Y REDES TELEINFORMÁTICAS
  • Ingeniería Administrativa
  • Ingeniería Industrial
  • LICENCIATURA EN ALIMENTACION Y NUTRICION
  • Licenciatur

## 4.3. PRUEBAS!

In [23]:
resultado = consultar_rag(
    query="¿Qué materias de matemáticas hay?",
    carreras_recomendadas=None
)

print(resultado["respuesta"])

 Base vectorial cargada: 249 vectores
 Busqueda inicial: 8 candidatos
   Chunk 50: Licenciatura en Psicopedagogía - Score: 20.9276
   Chunk 46: Licenciatura en Psicopedagogía - Score: 20.9326
   Chunk 45: Licenciatura en Psicopedagogía - Score: 21.5221
   Chunk 51: Licenciatura en Psicopedagogía - Score: 21.5348
   Chunk 95: Licenciatura en Medios de Comunicación e Informati - Score: 21.5699
   Chunk 44: Licenciatura en Psicopedagogía - Score: 21.8558
   Chunk 49: Licenciatura en Psicopedagogía - Score: 21.8623
   Chunk 48: Licenciatura en Psicopedagogía - Score: 21.9516
En la carrera de Licenciatura en Psicopedagogía, encontré una materia relacionada con matemáticas en el cuarto año - 8vo ciclo, que es "ESTADÍSTICA II" (Código: MA312) con 2 créditos. No hay más información disponible sobre otras materias de matemáticas en los pensums proporcionados.


In [24]:
resultado = consultar_rag(
    query="¿Qué materias de matemáticas tiene el pensum de INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN?",
    carreras_recomendadas=[
        "INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN",
        "INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS DE LA COMPUTACIÓN"
    ]
)

print(resultado["respuesta"])

 Base vectorial cargada: 249 vectores
 Busqueda inicial: 20 candidatos
   Chunk 148: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 12.4854
   Chunk 43: INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS  - Score: 12.5915
   Chunk 146: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 12.6209
   Chunk 95: Licenciatura en Medios de Comunicación e Informati - Score: 12.7595
   Chunk 238: INGENIERÍA EN TELECOMUNICACIONES Y REDES TELEINFOR - Score: 12.9031
   Chunk 42: INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS  - Score: 13.0208
   Chunk 142: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 13.1231
   Chunk 144: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 13.1810
   Chunk 129: Licenciatura en Comunicacion y Diseño - Score: 13.1948
   Chunk 234: INGENIERÍA EN TELECOMUNICACIONES Y REDES TELEINFOR - Score: 13.2016
   Chunk 41: INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS  - Score: 13.2366
   Chunk 50: Licenciatura en Psicopedagogía 

## 4.4 deepeval


In [41]:
deepeval_results = evaluar_rag_con_deepeval()



EVALUACIÓN DEL RAG CON DEEPEVAL

 Evaluando: ¿Qué materias de matemáticas tiene Ingeniería de Sistemas?
 Base vectorial cargada: 249 vectores
 Busqueda inicial: 20 candidatos
   Chunk 43: INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS  - Score: 18.0346
   Chunk 148: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 18.2013
   Chunk 147: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 18.8331
   Chunk 142: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 18.9015
   Chunk 42: INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS  - Score: 19.0214
   Chunk 234: INGENIERÍA EN TELECOMUNICACIONES Y REDES TELEINFOR - Score: 19.0938
   Chunk 144: INGENIERÍA DE SISTEMAS INFORMÁTICA Y CIENCIAS DE L - Score: 19.1198
   Chunk 37: INGENIERÍA EN ELECTRÓNICA, INFORMÁTICA Y CIENCIAS  - Score: 19.1983
   Chunk 50: Licenciatura en Psicopedagogía - Score: 19.3071
   Chunk 241: Licenciatura en Gastronomía y Administración Culin - Score: 19.4637
   Chunk 40: INGENIERÍA EN E

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4.1, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4.1, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4.1, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4.1, strict=False, async_mode=True)...

...

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ❌ Answer Relevancy (score: 0.6666666666666666, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 0.67 because the response did not provide the specific number of credits for the Ingeniería de Sistemas degree and instead included general offers of help, which are not directly relevant to the question. However, the response was polite and showed willingness to assist, which is why it is not scored lower., error: None)
  - ❌ Faithfulness (score: 0.5, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 0.50 because the actual output claims that information about the credits for specific cycles is not available, but the retrieval context does provide this information in the pensums., error: None)
  - ❌ Contextual Precision (score: 0.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 0.00 because both the first and second nodes in the retrieval contexts are irrelevant to the input. T

⚠ WARNING: No hyperparameters logged.
» ]8;id=949066;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 12.94s | token cost: 0.08907799999999999 USD)
» Test Results (3 total tests):
   » Pass Rate: 33.33% | Passed: 1 | Failed: 2

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.


RESULTADOS DE EVALUACION

--- Test Case 1 ---
Pregunta: ¿Qué materias de matemáticas tiene Ingeniería de Sistemas?
Metricas:
  • AnswerRelevancyMetric: N/A
  • FaithfulnessMetric: N/A
  • ContextualPrecisionMetric: N/A
  • ContextualRecallMetric: N/A

--- Test Case 2 ---
Pregunta: ¿Cuántos créditos tiene la carrera de Ingeniería de Sistemas?
Metricas:
  • AnswerRelevancyMetric: N/A
  • FaithfulnessMetric: N/A
  • ContextualPrecisionMetric: N/A
  • ContextualRecallMetric: N/A

--- Test Case 3 ---
Pregunta: ¿Qué materias de programación hay en el primer año?
Metricas:
  • AnswerRelevancyMetric: N/A
  • FaithfulnessMetric: N/A
  • ContextualPrecisionMetric: N/A
  • ContextualRecallMetric: N/A


## 4.5. Descargar la base de datos vectorial

este paso es para poderlo llevar posteriormente a produccion o la pagina web de Charmie

In [42]:
from google.colab import files
import shutil

# Comprimir la base vectorial
shutil.make_archive('vectordb', 'zip', 'data/vectordb')
files.download('vectordb.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>